<a href="https://colab.research.google.com/github/Lk0099/expert-fortnight/blob/main/neuralnetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
from keras.datasets import mnist
from keras.utils import to_categorical

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Preprocess the data
x_train = x_train.reshape(-1, 28*28) / 255.0
x_test = x_test.reshape(-1, 28*28) / 255.0
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Build the MLP model
model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(28*28,)))
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, batch_size=128, epochs=10, validation_data=(x_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test loss: {loss:.4f}")
print(f"Test accuracy: {accuracy:.4f}")


11490434/11490434 [==============================] - 0s 0us/step
Epoch 1/10
469/469 [==============================] - 6s 12ms/step - loss: 0.2703 - accuracy: 0.9230 - val_loss: 0.1370 - val_accuracy: 0.9611
Epoch 2/10
469/469 [==============================] - 6s 13ms/step - loss: 0.1029 - accuracy: 0.9687 - val_loss: 0.1033 - val_accuracy: 0.9697
Epoch 3/10
469/469 [==============================] - 5s 11ms/step - loss: 0.0686 - accuracy: 0.9789 - val_loss: 0.0864 - val_accuracy: 0.9721
Epoch 4/10
469/469 [==============================] - 3s 7ms/step - loss: 0.0492 - accuracy: 0.9844 - val_loss: 0.0745 - val_accuracy: 0.9773
Epoch 5/10
469/469 [==============================] - 4s 9ms/step - loss: 0.0373 - accuracy: 0.9882 - val_loss: 0.0739 - val_accuracy: 0.9772
Epoch 6/10
469/469 [==============================] - 3s 7ms/step - loss: 0.0281 - accuracy: 0.9909 - val_loss: 0.0718 - val_accuracy: 0.9787
Epoch 7/10
469/469 [==============================] - 3s 7ms/step - loss: 0.0204

In [ ]:
import cv2
import numpy as np

# Load pre-trained model for object detection
#net = cv2.dnn.readNetFromCaffe("deploy.prototxt", "model.caffemodel")

# Read video
video_path = "/content/WIN_20230708_16_42_33_Pro.mp4"
cap = cv2.VideoCapture(video_path)

# Define colors for bounding boxes
colors = np.random.uniform(0, 255, size=(80, 3))

# Process frames
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Perform object detection on the frame
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 0.007843, (300, 300), 127.5)
    net.setInput(blob)
    detections = net.forward()

    # Iterate over detected objects
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            class_id = int(detections[0, 0, i, 1])
            class_name = classes[class_id]

            # Get bounding box coordinates
            box = detections[0, 0, i, 3:7] * np.array([width, height, width, height])
            (startX, startY, endX, endY) = box.astype("int")

            # Draw bounding box and label on the frame
            color = colors[class_id]
            cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
            label = f"{class_name}: {confidence:.2f}"
            y = startY - 15 if startY - 15 > 15 else startY + 15
            cv2.putText(frame, label, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Display the resulting frame
    cv2.imshow("Object Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close windows
cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2

# Load pre-trained model for object detection
#net = cv2.dnn.readNetFromCaffe("deploy.prototxt", "model.caffemodel")

# Initialize object tracker
tracker = cv2.TrackerCSRT_create()

# Read video
video_path = "input_video.mp4"
cap = cv2.VideoCapture(video_path)

# Define colors for bounding boxes
colors = np.random.uniform(0, 255, size=(80, 3))

# Initialize variables
bbox = None
tracking_started = False

# Process frames
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    if not tracking_started:
        # Perform object detection on the first frame
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 0.007843, (300, 300), 127.5)
        net.setInput(blob)
        detections = net.forward()

        # Iterate over detected objects and select one for tracking
        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.5:
                box = detections[0, 0, i, 3:7] * np.array([width, height, width, height])
                (startX, startY, endX, endY) = box.astype("int")
                bbox = (startX, startY, endX - startX, endY - startY)
                tracker.init(frame, bbox)
                tracking_started = True
                break

    else:
        # Update tracker with the new frame
        success, bbox = tracker.update(frame)

        # Draw bounding box on the frame
        if success:
            (x, y, w, h) = [int(v) for v in bbox]
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow("Object Tracking", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close windows
cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2

# Load pre-trained model for object detection
net = cv2.dnn.readNetFromCaffe("deploy.prototxt", "model.caffemodel")

# Initialize object tracker
tracker = cv2.TrackerCSRT_create()

# Read video
video_path = "/content/WIN_20230708_16_42_33_Pro.mp4"
cap = cv2.VideoCapture(video_path)

# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_video = cv2.VideoWriter("output_video.avi", fourcc, fps, (width, height))

# Initialize variables
bbox = None
tracking_started = False

# Process frames
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    if not tracking_started:
        # Perform object detection on the first frame
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 0.007843, (300, 300), 127.5)
        net.setInput(blob)
        detections = net.forward()

        # Iterate over detected objects and select one for tracking
        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.5:
                box = detections[0, 0, i, 3:7] * np.array([width, height, width, height])
                (startX, startY, endX, endY) = box.astype("int")
                bbox = (startX, startY, endX - startX, endY - startY)
                tracker.init(frame, bbox)
                tracking_started = True
                break

    else:
        # Update tracker with the new frame
        success, bbox = tracker.update(frame)

        # Draw bounding box on the frame
        if success:
            (x, y, w, h) = [int(v) for v in bbox]
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Write frame with bounding box to output video
    output_video.write(frame)

    # Display the resulting frame
    cv2.imshow("Object Tracking", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close windows
cap.release()
output_video.release()
cv2.destroyAllWindows()


error: ignored